Section 1:	Write a python module to periodically scan and parse news from newswire website.

In [1]:
# Group-Number:3
# Group-Members:Jainam, Siddhant,Chaitali
# Course code:AIDI 1100
# Submission Date:07-04-2021
# Description of the program
# First we will perform webscraping

#Importing the essential modules
#Pandas for data analysis
import pandas as pd 

#Webpage parsing --> use package BS4, function called BeautifulSoup
from bs4 import BeautifulSoup as soup

#Web-client to grab the HTML page --> use package URLLIB, module called REQUEST, function called URLOPEN
import urllib
from urllib.request import urlopen as uReq

#For regular expression operations
import re

#Importing module for database
import sqlite3

#Date Time Module
import datetime

In [2]:
# importing custom module

from google.colab import drive

drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/colab')

import date_initializing

Mounted at /content/drive


In [3]:
#Create a database file
conn = sqlite3.connect('newsData.db', timeout=100)

#Create a cursor object, which lets us interact with the database using SQL commands
c = conn.cursor()

#Create a table in the database
try:
  c.execute('''Create TABLE newsparsemaster(npm_date TEXT, npm_web_page_text TEXT)''')
except:
  print("Table is Already Created")

In [4]:
#calling function from custom module for assigning date 
newsdates, newsmonth, newsday, newsyear = date_initializing.date()

2.	Keep track/store the already parsed news:

In [5]:
#-------------------------------------------Scan and parse the prnewswire webpage html-----------------------------------------------------------------------
dt=0
while dt<7:
  news_Url=""

  #Now generating the new url for web scraping symbols
  news_Url='https://www.prnewswire.com/news-releases/news-releases-list/?month='+str(newsmonth[dt])+'&day='+str(newsday[dt])+'&year='+str(newsyear[dt])+'&hour=00&page=1&pagesize=100'

  #To Handle  HTTP Error 403: Forbidden,Assign the headers with url in uReq
  header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}


  #Open the connection to the webpage
  uClient = uReq(urllib.request.Request(news_Url, headers=header))

  #Read the entire page, parse, and store into a variable
  news_page_html = uClient.read()

  #Close the connection to the news webpage
  uClient.close()

  #Parse html page to extract the data
  print(news_page_html)

  #Using beautiful soup to parse html to text
  page_soup = soup(news_page_html, 'html.parser')
  web_page_text = page_soup.get_text()

  #Store the data into the database
  c.execute('''INSERT INTO newsparsemaster VALUES(?,?)''', (newsdates[dt], web_page_text))
  dt=dt+1
#----------------------------------------------------------stored the data

b' <!doctype html>\n<html lang="en">\n<head>\n<meta charset="utf-8" />\n<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible" />\n<meta name="viewport" content="width=device-width, initial-scale=1" />\n<link rel="icon" href="/content/dam/prnewswire/icons/2019-Q4-PRN-Icon-32-32.png" type="image/png" />\n<meta name="adobeLaunchScript" content="//assets.adobedtm.com/launch-ENb957f6e7383d4549b191e1190ed5476e.min" /> <meta name="optanonhelper" content="us-prod" />\n<link rel="stylesheet" href="/etc.clientlibs/prncom/clientlibs/clientlibs-common.min.css" type="text/css">\n<script type="text/javascript" src="/etc.clientlibs/prncom/clientlibs/clientlib-library.min.js"></script>\n<script type="text/javascript" src="/etc.clientlibs/prncom/clientlibs/component-clientlibs.min.js"></script>\n<script type="text/javascript" src="https://widgets.financialcontent.com/cision/stockquote.js"></script>\n<meta name="description" content="Take a look at All News Releases related news releases, photos

In [6]:
#-------------------------------------------Retrive the data from the database
c.execute('''SELECT * FROM newsparsemaster''')

#Fetch the data into a Python variable
results = c.fetchall()
length = len(results)
for i in range(length):
    print(results[i])
#------------------------------------------------------------------------------    

('2021-04-13', ' \n\n\n\n\n\n\n \n\n\n\n\n All News Releases and Press Releases from PR Newswire  \n  \n \n\n\n\n\n\n\n \n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\nResources\n \nBlog\n\n\nJournalists\n\n\n\nLog In\n \nSign Up\n \nData Privacy\n\n\nSend a Release\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNews\n\n\nProducts \n\n\n\nOverview\n\n\nDistribution by PR Newswire\n\n\nCision Communications Cloud®\n\n\nCision IR\n\n\nAll Products\n\n\n\n\nContact \n\n\n\nGeneral Inquiries\n\n\nRequest a Demo\n\n\nEditorial Bureaus\n\n\nPartnerships\n\n\nMedia Inquiries\n\n\nWorldwide Offices\n\n\n\n\n\n\n\n\t\t\tvar siteSearchProperties = \'{"viewAllorglabelHtml":"Organizations <span>View all</span>","viewAllprodlabelHtml":null,"viewAllresourcelabelHtml":null,"newsReleasePrefix":null,"viewAllNewslabelHtml":null}\';\n\t\t\tsiteSearchProperties = JSON.parse(siteSearchProperties);\n\t\t\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\n\n\n\n\nSearch\nWhen typing in this field, a list of search results will appear and

In [7]:
#extracting stock symbol
url_stock_symbol = []
url_datewise_stock_symbol =[]
import random
for r in results :
 regularExpression=r"\([A-Z]+\s*:\s*([A-Z]{2,})\)"
  #finditer unpacks all matching substrings into a list and returns the length of it as well.
 matches = re.finditer(regularExpression, str(r))
#matchNum gives the match number of the list   
#match gives string which fulfill regular expression and value of starting and ending of the matching substring from the result string
#enumerate auto increment the start value
 for matchNum, match in enumerate(matches, start=1):
       #Remove comment to see all the stock symbol 
       #print ("{matchNum} Stock symbol {match}".format(matchNum = matchNum,match = match.group()))#Used custom format for better view
      
       for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        url_stock_symbol.append(match.group(groupNum))
 #print(url_stock_symbol[random.randint(0,10)])     
 url_datewise_stock_symbol.append(url_stock_symbol[random.randint(0,10)])

Auto_Selected_Stock=lambda url_datewise_stock_symbol=url_datewise_stock_symbol: print('Random selected each day"s stock symbol {0}'.format(url_datewise_stock_symbol))

       

3.	Prepare a nice visualization showing stock prices for last 10 days. 

In [9]:
#Visualization of the data as timeseries
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as gol

def plot():
  bsh = soup(uReq(url).read(), 'html.parser')
  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(go.Candlestick(x=df['Date'],
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close*'],visible = "legendonly"),
               secondary_y=True)
 
  fig.update_layout( title={
          'text': "Stock : "+bsh.h1.text,
          'y':0.9,
          'x':0.5,
          'xanchor': 'center',
          'yanchor': 'top'},xaxis_title="Dates",yaxis_title="Stock Close Price and Volume",xaxis_rangeslider_visible=True)
  fig.update_layout(legend=dict(
      yanchor="top",
      y=0.99,
      xanchor="left",
      x=0.01
  ))
  fig.add_trace(go.Scatter(name="Close",x=df['Date'],y=df['Close*'],line=dict(color='royalblue', width=0.5)),
                secondary_y=True)
  #To represnt the graphs in different colors
  co = "#"+"%06x" % random.randint(0, 0xFFFFFF)
  
  fig.add_trace(go.Bar(name="Volume",x=df['Date'],y=df['Volume'],marker_color=co), secondary_y=False)
  fig.show()

4.	Scan Yahoo! Finance page for the stock symbol appeared in the news. 

In [10]:
import requests 
Auto_Selected_Stock()
for udss in url_datewise_stock_symbol:
 url_sub=udss

 url = 'https://finance.yahoo.com/quote/'+url_sub+'/history?p='+url_sub+''
 r = requests.get(url)
 df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
 df = df_list[0]

#inserting first 10 values from the table present on website 
 df=df.head(10)

#to prevent the SettingWithCopyWarning
 pd.options.mode.chained_assignment = None

# To remove the null values from the data frame
 df.dropna(inplace=True)
 plot()

Random selected each day"s stock symbol ['MIK', 'PBR', 'ABG', 'XL', 'MIK', 'LDOS', 'XL']
